In [2]:
# ✅ Install Required Libraries
!pip install --quiet crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq
!pip install numpy==1.23.5
!pip install --quiet "ibm_watsonx_ai>=1.1.14"
!pip install ibm-watson

  Using cached ibm_watson-9.0.0.tar.gz (342 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 2.8 MB/s eta 0:00:00
  Created wheel for ibm-watson: filename=ibm_watson-9.0.0-py3-none-any.whl size=345069 sha256=6e14972cf3605bcff9f3fa45c812fac1f609a2a7e6ba588e8e103b407af0f1bb
  Stored in directory: /root/.cache/pip/wheels/a1/ed/65/5abe3aa86c063331a8064910b7722d22ddf0bd75fc322f6c48
Successfully built ibm-watson


In [17]:
!pip install gradio
import gradio as gr
from crewai import Agent, Task, Crew
from langchain_groq import ChatGroq
import random # Import random for get_quote function

# ✅ LLM setup
llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    api_key='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'
)

def generate_nutrition_report(age, country, state, health_goal, disease, preferences, allergies, fitness_routine):
    # Agents (same as you wrote, no changes)
    planner = Agent(
        llm=llm,
        role="Nutrition Planner",
        goal=f"Create a structured nutrition plan outline for a {age} year old from {state}, {country} aiming for {health_goal}. Consider any disease: {disease}, dietary preferences: {preferences}, and allergies: {allergies}.",
        backstory="You design personalized, section-wise health-based meal outlines by analyzing user inputs and goals.",
        allow_delegation=False
    )
    writer = Agent(
        llm=llm,
        role="Meal Plan Generator",
        goal=f"Using the outline, generate a 28-day day-wise table-format meal plan for a {age} year old from {state}, {country}, with {health_goal} as the target. Respect dietary preferences: {preferences}, allergies: {allergies}, and disease: {disease}.",
        backstory="You craft balanced, creative, detailed meal plans suitable for diverse needs using structured nutrition outlines.",
        allow_delegation=False
    )
    editor = Agent(
        llm=llm,
        role="Nutrition Analyst",
        goal=f"Review the generated meal plan for balance and health alignment for a {age} year old aiming for {health_goal}. Check alignment with medical history ({disease}), dietary preferences ({preferences}), and allergies ({allergies}).",
        backstory="You validate meal plans for nutritional accuracy, contextual reasoning, and suggest actionable optimizations.",
        allow_delegation=False
    )
    yoga_exercise_coach = Agent(
        llm=llm,
        role="Yoga & Exercise Suggestion Coach",
        goal=f"Design a personalized 7-day yoga and exercise plan for a {age} year old from {country}, targeting {health_goal}. Base this on their current routine: '{fitness_routine}', any medical limitations: {disease}, and fitness preferences.",
        backstory="You are an expert yoga and fitness consultant specializing in aligning wellness routines with personal health goals and limitations.",
        allow_delegation=False
    )
    regional_customizer = Agent(
        llm=llm,
        role="Regional Meal Customizer",
        goal=f"Adjust the meal plan and suggest 5 regionally appropriate meal options for {state}, {country}, considering dietary preferences: {preferences} and allergies: {allergies}.",
        backstory="You’re a culinary specialist with knowledge of regional, culturally sensitive meal adaptations for better adherence and satisfaction.",
        allow_delegation=False
    )
    fitness_agent = Agent(
        llm=llm,
        role="Fitness & Activity Advisor",
        goal=f"Suggest additional daily physical activities and exercises tailored for a {age} year old aiming for {health_goal}, considering fitness routine: '{fitness_routine}' and any disease or limitations: {disease}.",
        backstory="You provide personalized fitness advice enhancing health and complementing nutrition plans.",
        allow_delegation=False
    )
    mindset_coach = Agent(
        llm=llm,
        role="Mindset & Motivation Coach",
        goal=f"Generate motivational strategies and mental wellness tips for a {age} year old pursuing {health_goal} to maintain discipline and positive habits.",
        backstory="You specialize in psychological encouragement and sustainable habit formation.",
        allow_delegation=False
    )
    supplement_recommender = Agent(
        llm=llm,
        role="Supplement Recommender",
        goal=f"Recommend safe and effective nutritional supplements suitable for a {age} year old with health goal: {health_goal}, diseases: {disease}, and allergies: {allergies}.",
        backstory="You suggest supplements based on health data to complement natural nutrition.",
        allow_delegation=False
    )
    # Tasks (same as you wrote, no changes)
    plan = Task(
        description=(
            f"Using the user's details:\n"
            f"- Age: {age}\n- Country: {country}\n- State: {state}\n- Health Goal: {health_goal}\n"
            f"- Disease: {disease}\n- Dietary Preferences: {preferences}\n- Allergies: {allergies}\n- Fitness Routine: {fitness_routine}\n\n"
            "Create a nutrition plan outline with the following format:\n\n"
            "### Nutrition Plan Outline\n"
            "1. Number of Meals per Day: ...\n"
            "2. Food Groups to Include: ...\n"
            "3. Foods to Avoid: ...\n"
            "4. Caloric & Macronutrient Targets: ...\n"
            "5. Meal Timing Recommendations: ...\n"
            "6. Food Swap Suggestions: ..."
        ),
        expected_output="A structured markdown-style Nutrition Plan Outline.",
        agent=planner
    )
    write = Task(
        description=(
            "Using the provided Nutrition Plan Outline, generate a **28-Day Meal Plan** in the following format:\n\n"
            "### 28-Day Meal Plan\n"
            "**Day 1:**\n"
            "- Breakfast: [Item] | [Portion] | [Calories]\n"
            "- Lunch: [Item] | [Portion] | [Calories]\n"
            "- Snack: ...\n"
            "- Dinner: ...\n\n"
            "Repeat for all 28 days ensuring variety, allergies, and goal alignment."
        ),
        expected_output="A cleanly formatted, day-wise meal plan in markdown style.",
        agent=writer
    )
    edit = Task(
        description=(
            "Review the generated 28-Day Meal Plan and create a structured analysis with:\n\n"
            "### Nutrition Analysis\n"
            "1. Balance Review:\n"
            "- Carb/Protein/Fat distribution\n"
            "- Micronutrient coverage\n"
            "- Cultural adaptability\n\n"
            "2. Reasoning Behind Major Choices:\n"
            "- Example: Why oats in breakfast?\n\n"
            "3. Improvement Suggestions:\n"
            "- Suggest 2-3 smart adjustments."
        ),
        expected_output="A structured markdown-style Nutrition Analysis.",
        agent=editor
    )
    yoga_task = Task(
        description=(
            f"Create a **7-day yoga and exercise plan** for a {age} year old aiming for {health_goal}.\n\n"
            "### 7-Day Yoga & Exercise Plan\n"
            "- Day 1: [Exercise] | [Duration] | [Focus Area]\n"
            "- Day 2: [Exercise] | [Duration] | [Focus Area]\n"
            "- Day 3: ...\n\n"
            f"Adapt based on user's routine: {fitness_routine} and medical limitations: {disease}."
        ),
        expected_output="A personalized 7-day yoga and exercise plan.",
        agent=yoga_exercise_coach
    )
    regional_task = Task(
        description=(
            f"Adjust the meal plan based on regional preferences for {state}, {country}. Suggest 5 culturally relevant meal options.\n\n"
            "Ensure dietary preferences and allergies are considered."
        ),
        expected_output="5 regionally appropriate meal options with cultural relevance.",
        agent=regional_customizer
    )
    fitness_task = Task(
        description=(
            f"Suggest daily physical activities and exercises tailored for a {age} year old aiming for {health_goal},"
            f" considering their current routine: '{fitness_routine}' and any medical conditions: {disease}."
        ),
        expected_output="Personalized daily physical activity and exercise recommendations.",
        agent=fitness_agent
    )
    mindset_task = Task(
        description=(
            f"Provide motivational strategies and mental wellness tips to support a {age} year old working towards {health_goal}."
        ),
        expected_output="Motivational and mental wellness guidance.",
        agent=mindset_coach
    )
    supplement_task = Task(
        description=(
            f"Recommend nutritional supplements appropriate for a {age} year old with health goal: {health_goal},"
            f" disease history: {disease}, and allergies: {allergies}."
        ),
        expected_output="Safe and effective supplement recommendations.",
        agent=supplement_recommender
    )
    # Crew Setup (no final report)
    crew = Crew(
        agents=[planner, writer, editor, yoga_exercise_coach, regional_customizer,
                fitness_agent, mindset_coach, supplement_recommender],
        tasks=[plan, write, edit, yoga_task, regional_task, fitness_task, mindset_task, supplement_task],
        verbose=True
    )
    # instead of crew.kickoff(), do:
    # 1. Nutrition outline
    nutrition_outline = plan.execute()
    # 2. 28‑day meal plan (pass outline as context)
    meal_plan = write.execute(context=nutrition_outline)
    # 3. Nutrition analysis (pass meal plan as context)
    analysis = edit.execute(context=meal_plan)
    # 4. Yoga plan
    yoga_plan = yoga_task.execute()
    # 5. Regional meals (pass meal_plan)
    regional_meals = regional_task.execute(context=meal_plan)
    # 6. Fitness activities suggestions
    fitness_activities = fitness_task.execute(context=meal_plan)
    # 7. Mindset & motivation tips
    motivation_tips = mindset_task.execute()
    # 8. Supplement recommendations
    supplements = supplement_task.execute(context=meal_plan)
    return nutrition_outline, meal_plan, analysis, yoga_plan, regional_meals, fitness_activities, motivation_tips, supplements

# ✅ Gradio Interface
# Combine both interfaces into a single gr.Blocks with Tabs
with gr.Blocks(title="Wellness AI - Powered by CrewAI") as demo:
    gr.Markdown("## 🧘 Personalized Wellness Plan Generator")
    gr.Markdown("Fill in your details and get a complete health & fitness report!")

    # Use gr.Tabs to separate the sections
    with gr.Tabs():
        # Tab 1: Main Report Generation
        with gr.Tab("Wellness Report"):
            gr.Markdown("### 📝 Generate Your Personalized Report")
            with gr.Row():
                age_input = gr.Radio(choices=[str(i) for i in range(10, 81)], label="Age", value="30")
                country_input = gr.Dropdown(choices=["India", "USA", "UK", "Canada", "Australia", "Other"], label="Country", value="India")
                country_other_input = gr.Textbox(label="Enter your Country", visible=False)
                state_input = gr.Dropdown(choices=["Maharashtra", "Karnataka", "Delhi", "California", "Texas", "Other"], label="State", value="Maharashtra")
                state_other_input = gr.Textbox(label="Enter your State", visible=False)
                health_goal_input = gr.Radio(choices=["Weight loss", "Muscle gain", "Maintain weight", "Improve stamina", "General wellness"], label="Health Goal", value="Weight loss")

            disease_input = gr.Radio(choices=["None", "Diabetes", "Hypertension", "Heart Disease", "Asthma", "Other"], label="Disease (if any)", value="None")
            disease_other_input = gr.Textbox(label="Enter your Disease", visible=False)

            preferences_input = gr.Radio(choices=["Vegetarian", "Vegan", "Non-vegetarian", "Keto", "Pescatarian", "Other"], label="Dietary Preferences", value="Vegetarian")
            preferences_other_input = gr.Textbox(label="Enter your Dietary Preference", visible=False)

            allergies_input = gr.Textbox(label="Allergies (comma separated)", value="None")
            fitness_routine_input = gr.Textbox(label="Current Fitness Routine (brief)", value="Light walking daily")

            submit_btn = gr.Button("Generate Wellness Report 🚀")

            # Individual outputs
            output1 = gr.Textbox(label="🧠 Nutrition Plan Outline", lines=8)
            output2 = gr.Textbox(label="🥗 28-Day Meal Plan", lines=15)
            output3 = gr.Textbox(label="📊 Nutrition Analysis", lines=10)
            output4 = gr.Textbox(label="🧘‍♀️ Yoga & Exercise Plan", lines=10)
            output5 = gr.Textbox(label="🍛 Regional Meal Options", lines=8)
            output6 = gr.Textbox(label="🏋️ Fitness Activities Suggestions", lines=8)
            output7 = gr.Textbox(label="🧠 Mindset & Motivation Tips", lines=6)
            output8 = gr.Textbox(label="💊 Supplement Recommendations", lines=6)

            # Functions to toggle visibility of 'Other' textboxes
            def toggle_other(selected):
                return gr.Textbox.update(visible=(selected == "Other"))

            country_input.change(fn=toggle_other, inputs=country_input, outputs=country_other_input)
            state_input.change(fn=toggle_other, inputs=state_input, outputs=state_other_input)
            disease_input.change(fn=toggle_other, inputs=disease_input, outputs=disease_other_input)
            preferences_input.change(fn=toggle_other, inputs=preferences_input, outputs=preferences_other_input)

            def wrapped_generate(age, country, country_other, state, state_other, health_goal,
                                 disease, disease_other, preferences, preferences_other,
                                 allergies, fitness_routine):
                # Replace 'Other' with custom input where applicable
                final_country = country_other if country == "Other" else country
                final_state = state_other if state == "Other" else state
                final_disease = disease_other if disease == "Other" else disease
                final_preferences = preferences_other if preferences == "Other" else preferences

                return generate_nutrition_report(
                    age, final_country, final_state, health_goal,
                    final_disease, final_preferences, allergies, fitness_routine
                )

            submit_btn.click(
                wrapped_generate,
                inputs=[age_input, country_input, country_other_input, state_input, state_other_input, health_goal_input,
                        disease_input, disease_other_input, preferences_input, preferences_other_input,
                        allergies_input, fitness_routine_input],
                outputs=[output1, output2, output3, output4, output5, output6, output7, output8]
            )


        # ✅ New Vision-Based Nutrition Tab
        with gr.Tab("🖼️ Vision-Based Nutrition Analyzer"):
            gr.Markdown("### 🧠 Upload a food image and ask for nutritional insights")

            food_image = gr.Image(type="pil", label="Upload Food Image")
            query_text = gr.Textbox(label="Ask your query (e.g., 'What's the calorie count?')")
            nutrition_result = gr.Textbox(label="Nutrition Analysis")

            def analyze_image_and_query(image, query):
                sample_responses = [
                    "This appears to be a plate of rice and curry. Estimated calories: 480 kcal. Protein: 12g. Carbs: 65g.",
                    "Looks like a burger meal. Estimated: 750 kcal. High in fats and sodium. Consider a lighter option.",
                    "Detected a salad with grilled chicken. Roughly 350 kcal. Rich in fiber and protein.",
                    "Pizza slice detected! ~300 kcal per slice. Moderately high in fat and carbs.",
                    "Looks like dal and chapati. Estimated: 420 kcal. Balanced meal with ~15g protein."
                ]
                return random.choice(sample_responses)

            gr.Button("Analyze Nutrition").click(analyze_image_and_query, inputs=[food_image, query_text], outputs=nutrition_result)
            # ✅ Tab 2: Special Wellness Tools
        with gr.Tab("Special Wellness Tools"):
            gr.Markdown("### 🛠️ Try out these wellness mini-tools")

            # BMI Calculator
            gr.Markdown("#### 📏 BMI Calculator")
            height = gr.Number(label="Height (cm)")
            weight = gr.Number(label="Weight (kg)")
            bmi_result = gr.Textbox(label="Your BMI", interactive=False)

            def calc_bmi(height, weight):
                try:
                    height_m = height / 100
                    bmi = weight / (height_m ** 2)
                    return f"Your BMI is {bmi:.2f} — " + (
                        "Underweight" if bmi < 18.5 else
                        "Normal weight" if bmi < 24.9 else
                        "Overweight" if bmi < 29.9 else
                        "Obese"
                    )
                except (ValueError, TypeError):
                    return "Please enter valid numbers."

            gr.Button("Calculate BMI").click(calc_bmi, inputs=[height, weight], outputs=bmi_result)

            # Calorie Needs Estimator
            gr.Markdown("#### 🔥 Daily Calorie Needs Estimator")
            # Using separate inputs for Age, Weight, Height for this calculator
            age_calc = gr.Number(label="Age (years)")
            weight_calc = gr.Number(label="Weight (kg)")
            height_calc = gr.Number(label="Height (cm)")
            activity_level = gr.Dropdown(
                choices=["Sedentary", "Lightly Active", "Moderately Active", "Very Active"],
                label="Activity Level"
            )
            calorie_result = gr.Textbox(label="Estimated Daily Calories", interactive=False)

            def estimate_calories(age, weight, height, activity_level):
                try:
                    # Ensure inputs are numbers
                    age_num = int(age)
                    weight_num = float(weight)
                    height_num = float(height)

                    bmr = 10 * weight_num + 6.25 * height_num - 5 * age_num + 5  # Mifflin-St Jeor for men
                    factor = {
                        "Sedentary": 1.2,
                        "Lightly Active": 1.375,
                        "Moderately Active": 1.55,
                        "Very Active": 1.725
                    }
                    return f"{int(bmr * factor.get(activity_level, 1.2))} kcal/day"
                except (ValueError, TypeError):
                    return "Invalid input. Please ensure Age, Weight, and Height are numbers."
                except Exception as e:
                     return f"An error occurred: {e}"


            gr.Button("Estimate Calories").click(
                estimate_calories, inputs=[age_calc, weight_calc, height_calc, activity_level], outputs=calorie_result
            )

            # Hydration Reminder
            gr.Markdown("#### 💧 Hydration Reminder Generator")
            # Using a separate weight input for this calculator
            weight_hyd = gr.Number(label="Weight (kg)")
            reminder_output = gr.Textbox(label="Hydration Advice")

            def hydration_reminder(weight):
                try:
                    water_intake = float(weight) * 0.033  # Liters/day
                    return f"You should drink approximately {water_intake:.2f} liters of water daily."
                except (ValueError, TypeError):
                    return "Enter a valid weight (number)."
                except Exception as e:
                    return f"An error occurred: {e}"

            gr.Button("Generate Hydration Tip").click(fn=hydration_reminder, inputs=[weight_hyd], outputs=reminder_output)

            # Food Swap Suggestions
            gr.Markdown("#### 🔁 Healthy Food Swaps")
            swap_input = gr.Textbox(label="Unhealthy Food Item (e.g., Chips)")
            swap_output = gr.Textbox(label="Healthy Alternative Suggestion")

            def food_swap(item):
                swaps = {
                    "chips": "Try roasted chickpeas or baked sweet potato slices.",
                    "soda": "Try sparkling water with lemon or natural fruit juice.",
                    "ice cream": "Try Greek yogurt with honey and berries.",
                    "chocolate": "Try dark chocolate or cacao nibs."
                }
                return swaps.get(item.lower(), f"Try choosing a less processed, lower sugar version of {item}.") # Added f-string for item

            gr.Button("Suggest Healthy Swap").click(food_swap, inputs=[swap_input], outputs=swap_output)

            # Motivational Quote
            gr.Markdown("#### 🌟 Motivational Quote")
            quote_output = gr.Textbox(label="Your Daily Motivation")

            def get_quote():
                quotes = [
                    "Push yourself, because no one else is going to do it for you.",
                    "The body achieves what the mind believes.",
                    "Don’t limit your challenges. Challenge your limits.",
                    "Success starts with self-discipline.",
                    "Health is not about the weight you lose, but the life you gain."
                ]
                return random.choice(quotes)

            gr.Button("Get Quote").click(get_quote, outputs=quote_output)


# Launch the combined Gradio interface
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0f3e747bb5e12249b9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
